### Imports

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
import copy, os, math, time
import matplotlib.pyplot as plt

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Tokenizer

In [2]:
class Tokenizer:
    def __init__(self, vocab_size_original, vocab_size_target, ):
        self.vocab_size_original = vocab_size_original
        self.vocab_size_target = vocab_size_target
        self.num_merges = vocab_size_target - vocab_size_original
        self.merges = {}

    def fit(self, tokens):

        for i in range(self.num_merges):
            pair_freq = self.get_pair_freq(tokens)
            most_frequent = max(pair_freq, key=pair_freq.get)
            idx = self.vocab_size_original + i
            tokens = self.merge(tokens, most_frequent, idx)
            self.merges[most_frequent] = idx

        self.vocab = {i: bytes([i]) for i in range(self.vocab_size_original)}
        for pair, idx in self.merges.items():
            self.vocab[idx] = self.vocab[pair[0]] + self.vocab[pair[1]]
        
        return self

    def get_pair_freq(self, text):
        pair_freq = {}
        for i in range(len(text) - 1):
            tok1 = text[i]
            tok2 = text[i + 1]

            if (tok1, tok2) in pair_freq:
                pair_freq[(tok1, tok2)] += 1
            else:
                pair_freq[(tok1, tok2)] = 1
        return pair_freq

    def merge(self, tokens, pair, idx):
        new_ids = []
        i = 0
        while i < len(tokens):
            if i < len(tokens) - 1 and (tokens[i], tokens[i + 1]) == pair:
                new_ids.append(idx)
                i += 2
            else:
                new_ids.append(tokens[i])
                i += 1
        return new_ids

    def encode(self, text):
        tokens = list(text.encode("utf-8"))
        while len(tokens) >= 2:
            pair_freq = self.get_pair_freq(tokens)
            pair = max(pair_freq, key=lambda p: self.merges.get(p, float("-inf")))
            if pair not in self.merges:
                break
            idx = self.merges[pair]
            tokens = self.merge(tokens, pair, idx)
        return tokens

    def decode(self, tokens):
        tokens = b"".join(self.vocab[i] for i in tokens)
        text = tokens.decode("utf-8", errors="replace")
        return text
    

# Transformer

In [3]:
def clone(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class Embeddings(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        super().__init__()
        self.token_embeddings = nn.Embedding(vocab_size, embedding_size)
        self.embedding_size = embedding_size
    
    def forward(self, x):
        return self.token_embeddings(x) * math.sqrt(self.embedding_size)
    
class PositionalEncoding(nn.Module):
    def __init__(self, embedding_size, dropout, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

        pe = torch.zeros(max_len, embedding_size)
        position = torch.arange(0, max_len).unsqueeze(1)
        power = torch.exp(-torch.arange(0, embedding_size, 2) * math.log(10_000) / embedding_size)
        pe[:, 0::2] = torch.sin(position * power)
        pe[:, 1::2] = torch.cos(position * power)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + Variable(self.pe[:, : x.size(1)], requires_grad=False)
        return self.dropout(x)

class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super().__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super().__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

class Encoder(nn.Module):
    def __init__(self, layer, N):
        super().__init__()
        self.layers = clone(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super().__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clone(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask, need_weights=False)[0])
        return self.sublayer[1](x, self.feed_forward)
    
class Decoder(nn.Module):
    def __init__(self, layer, N):
        super().__init__()
        self.layers = clone(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)
    
class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super().__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clone(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, attn_mask=tgt_mask, need_weights=False, is_causal=True)[0])
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, key_padding_mask=src_mask, need_weights=False)[0])
        return self.sublayer[2](x, self.feed_forward)

def attention(query, key, value, mask=None, dropout=None):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

class MultiHeadAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super().__init__()
        assert d_model % h == 0
        self.d_k = d_model // h
        self.h = h
        self.linears = clone(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2) for l, x in zip(self.linears, (query, key, value))]
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

class Generator(nn.Module):
    def __init__(self, embedding_size, vocab_size):
        super().__init__()
        self.proj = nn.Linear(embedding_size, vocab_size)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)
    
def subsequent_mask(size):
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype("uint8")
    return torch.from_numpy(subsequent_mask) == 0 

def make_model(src_vocab, tgt_vocab, N=1, d_model=256, d_ff=512, h=4, dropout=0.1):
    c = copy.deepcopy
    attn = nn.MultiheadAttention(d_model, h, dropout=dropout, batch_first=True)
    #attn = MultiHeadAttention(h, d_model, dropout)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)

    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(src_vocab, d_model), c(position)),
        nn.Sequential(Embeddings(tgt_vocab, d_model), c(position)),
        Generator(d_model, tgt_vocab),
    )

    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    
    return model

## Training

In [4]:
class Batch:
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src == pad)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum().item()

    @staticmethod
    def make_std_mask(tgt, pad):
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask == 0
    
class MyIterator:
    def __init__(self, data, seq_len, batch_size, n_batches, device):
        self.data = data
        self.seq_len = seq_len
        self.batch_size = batch_size
        self.n_batches = n_batches
        self.i = 0
        self.device = device

    def __iter__(self):
        return self

    def __next__(self):
        if self.i >= self.n_batches:
            self.i = 0
            raise StopIteration
        else:
            self.i += 1
            ix = torch.randint(len(self.data) - self.seq_len, (self.batch_size,))
            x = torch.stack([self.data[i:i + self.seq_len] for i in ix])
            y = torch.stack([self.data[i + 1:i + self.seq_len+1] for i in ix])
            starts = torch.ones((self.batch_size, 1), dtype=y.dtype)
            y = torch.cat([starts, y], dim=1)
            return Batch(x.to(self.device), y.to(self.device), 0)


def run_epoch(data_iter, model, loss_compute):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src, batch.trg, 
                            batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens

class LabelSmoothing(nn.Module):
    def __init__(self, size, padding_idx, smoothing=0.0):
        super().__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None

    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

class SimpleLossCompute:
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt

    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1)) / norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.zero_grad()
        return loss.item() * norm

# Loading Data

In [ ]:
from transformers import AutoTokenizer
import spacy
from spacy.lang.en import English

nlp = English()
nlp.add_pipe("sentencizer")
nlp.max_length = 2_000_000

with open(os.path.join("data", "tiny-shakespeare.txt"), "r") as file:
    text = file.read()

text = nlp(text)

test = "Hello my name is john. Bob is my friend? He is 10 years old."
data_set = [x.text + "\n" for x in text.sents]

def get_training_corpus():
    for i in range(0, len(data_set), 100):
        yield data_set[i:i + 100]

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

vocab_size = 5_000
tokenizer = old_tokenizer.train_new_from_iterator(get_training_corpus(), vocab_size=vocab_size)

train_text = torch.tensor(tokenizer.encode("".join(data_set)))

TypeError: unsupported operand type(s) for +: 'spacy.tokens.span.Span' and 'str'

## Training

In [7]:
split = int(0.9 * len(train_text))
criterion = LabelSmoothing(size=vocab_size, padding_idx=0, smoothing=0.05)
model = make_model(vocab_size, vocab_size, N=2, d_model=512, d_ff=2048, h=4, dropout=0.1)
model_opt = torch.optim.Adam(model.parameters(), lr=1e-3)
data_iter_train = MyIterator(train_text[:split], 100, 20, 1500, DEVICE)
data_iter_valid = MyIterator(train_text[split:], 100, 20, 5, DEVICE)

model.to(DEVICE)
model.train()
run_epoch(data_iter_train, model, SimpleLossCompute(model.generator, criterion, model_opt))
model.eval()
print(run_epoch(data_iter_valid, model, SimpleLossCompute(model.generator, criterion, None)))

/home/codygrogan/.local/lib/python3.12/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch Step: 1 Loss: 6.047389 Tokens per Sec: 27558.795556
Epoch Step: 51 Loss: 4.936026 Tokens per Sec: 30680.067974
Epoch Step: 101 Loss: 4.917794 Tokens per Sec: 30451.946297
Epoch Step: 151 Loss: 4.770451 Tokens per Sec: 30151.428419
Epoch Step: 201 Loss: 4.726799 Tokens per Sec: 30147.744147
Epoch Step: 251 Loss: 4.650562 Tokens per Sec: 30133.524802
Epoch Step: 301 Loss: 4.395939 Tokens per Sec: 30029.340563
Epoch Step: 351 Loss: 4.204588 Tokens per Sec: 30083.781635
Epoch Step: 401 Loss: 3.966031 Tokens per Sec: 29973.969416
Epoch Step: 451 Loss: 3.874089 Tokens per Sec: 29149.171366
Epoch Step: 501 Loss: 3.743522 Tokens per Sec: 29962.120024
Epoch Step: 551 Loss: 3.893653 Tokens per Sec: 29871.736755
Epoch Step: 601 Loss: 3.653706 Tokens per Sec: 29902.409999
Epoch Step: 651 Loss: 3.366473 Tokens per Sec: 29809.629490
Epoch Step: 701 Loss: 3.531538 Tokens per Sec: 29583.546195
Epoch Step: 751 Loss: 3.469882 Tokens per Sec: 29048.076137
Epoch Step: 801 Loss: 3.510599 Tokens per S

In [9]:
def greedy_decode(model, src, src_mask, max_len, start_symbol, device):
    src = src.to(device)
    src_mask = src_mask.to(device)
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    model.train()
    for i in range(max_len -1):
        probs = []
        for j in range(4):
            out = model.decode(memory, src_mask, Variable(ys).to(device),
                            Variable(subsequent_mask(ys.size(1)) == 1).to(device))
            probs.append(torch.exp(model.generator(out[:, -1])))
        prob = torch.stack(probs, dim=0).mean(dim=0)
        next_word = torch.multinomial(prob, num_samples=1).squeeze(1)
        next_word = next_word.data[0]
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

model.eval()
text = "Hello?"
src = torch.tensor(tokenizer.encode(text)).unsqueeze(0)
src_mask = Variable(torch.ones(1, src.size(1)) == 0)
out = greedy_decode(model, src, src_mask, max_len=1000, start_symbol=1, device=DEVICE)
print(tokenizer.decode(out[0].cpu().numpy()))


!allWill
bedBut
thee:
hopegentlyon
?MENENIUS
:Nowarch
scope:
Catesbyere
shallwe
leave.
DORCAS
Whythat
wherewe
shallmytrust
.Second
shame.
LORD
:HORTENS
CAPULET:
Tomorrow
?GRUMIO
:An
bis
s.
havetheclouds
willI
theday
itupon
myhead
andthereineffect
.CAPULET
:Why
allthe
mymother
lady?
twenty
friendit
would
.
?
stoo
poorwidow
mustI
truePearethe
ifthou
fatherthat
welcome
fora
Oxford
togive
wonderhow
:
the
?lawand
llseeWhy
lack
go!
lord
lord
frightarry
wits
!
the
tellace
are
promisethatnestto
foesThusdo
in
whata
un
friends?
soundly?
?
?
!
needcommittedin
ghyousir
?
.
manyFirst
long
twenty
!
tellme
.
kinsman
drop
drivencuedmen
?
virtuous
livelyatfirstthat
ho!
wine.
meetagain
Asmy
?
cualtogether.
again
allthis
careto
this
lordswisewithyour
ofus
erlygoodmakes
?�
how
?
Citizen:
and
it
?
.
ourintedIsee
?
ofthis
but
inthis
andme
agentlemanadisemy
scurkswe
gainst
?niciousas
!
amongtitude?
gentle
never
inthe
mywife
yetthat
hollI
awayon
as
Iwill
willnot
kings
!
plant
how
apthat
?
wetwo.
thatI
First
i